##  (공공데이터분석) 국토부 아파트매매 실거래가 API 연동

#### XML 국토교통부 아파트매매 실거래자료 : https://www.data.go.kr/data/15058747/openapi.do

#### 행정표준관리시스템 : https://www.code.go.kr/index.do
#### ( [메뉴] 자주이용하는 코드 > 법정동 > 법정동 코드 전체자료 다운로드)

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

##### 데이터 로드

In [2]:
base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

In [3]:
area_cd = "LAWD_CD=" + "11680"    # 11680 : 강남구
deal_dt = "DEAL_YMD=" + "201912"
svc_key = "serviceKey=" + "fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D"

In [4]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=201912&serviceKey=fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D'

In [5]:
response = requests.get(url)
response

<Response [200]>

In [6]:
# response.text

##### 법정동 코드(지역코드) 가공

In [7]:
lawd_cd = pd.read_csv("data/법정동코드_전체자료.txt", sep='\t', encoding ='cp949') # encoding : 'euc-kr', 'utf-8'
lawd_cd

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


In [8]:
# 데이터 조작 편의성을 위한 Column 이름 변경
print(lawd_cd.columns)
lawd_cd.columns = ["code", "name", "check"] # Column 이름이 한글이면 코딩시 불편함
lawd_cd

Index(['법정동코드', '법정동명', '폐지여부'], dtype='object')


,code,name,check
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


In [9]:
lawd_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46292 entries, 0 to 46291
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    46292 non-null  int64 
 1   name    46292 non-null  object
 2   check   46292 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [10]:
in_name = "종로구"
tmp = lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")]#.head(1)
tmp

,code,name,check
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
5,1111010400,서울특별시 종로구 효자동,존재
...,...,...,...
84,1111018300,서울특별시 종로구 평창동,존재
85,1111018400,서울특별시 종로구 부암동,존재
86,1111018500,서울특별시 종로구 홍지동,존재
87,1111018600,서울특별시 종로구 신영동,존재


In [11]:
tmp = tmp.iloc[0,0]
str(tmp)
tmp = tmp.astype(str)
tmp[0:5]

'11110'

In [12]:
# 위를 정리
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]

'11110'

In [13]:
# 함수 생성 :
def find_lawd_cd(in_text):
    return lawd_cd[(lawd_cd['name'].str.contains(in_text)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]

In [14]:
in_name = "강남구"
find_lawd_cd(in_name)

'11680'

##### 데이터 값 직접 입력

In [15]:
in_lawd = input("검색지역 입력(예: 강남구) : ")

검색지역 입력(예: 강남구) : 서북구


In [16]:
in_deal_ymd = input("계약월 입력(예: 201912) : ")

계약월 입력(예: 201912) : 202212


In [17]:
in_svckey = "fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D"

In [18]:
in_lawd_cd = find_lawd_cd(in_lawd)
in_lawd_cd

'44133'

In [19]:
area_cd = "LAWD_CD=" + in_lawd_cd
deal_dt = "DEAL_YMD=" + in_deal_ymd
svc_key = "serviceKey=" + in_svckey

In [20]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=44133&DEAL_YMD=202212&serviceKey=fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D'

In [21]:
response = requests.get(url)
response

<Response [200]>

##### 데이터 전처리 (web data / API output)

In [22]:
soup = BeautifulSoup(response.text, "lxml")
# soup

C:\ProgramData\Anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [23]:
# soup.find_all('item')

In [24]:
import xml.etree.ElementTree as ET

def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list
    
items_list = find_xml_items(response)
items_list[1]

{'거래금액': '41,000',
 '거래유형': '중개거래',
 '건축년도': '2021',
 '년': '2022',
 '법정동': '와촌동',
 '아파트': '천안역사동아라이크텐',
 '월': '12',
 '일': '19',
 '전용면적': '84.9864',
 '중개사소재지': '충남 천안서북구',
 '지번': '186',
 '지역코드': '44133',
 '층': '38',
 '해제사유발생일': '',
 '해제여부': ''}

##### 결과

In [25]:
result = pd.DataFrame(items_list)
print(result.shape)
result.head()

(170, 15)


,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"29,700",중개거래,2021,2022,와촌동,천안역사동아라이크텐,12,5,65.4418,충남 천안서북구,186,44133,11,,
1,"41,000",중개거래,2021,2022,와촌동,천안역사동아라이크텐,12,19,84.9864,충남 천안서북구,186,44133,38,,
2,"41,000",중개거래,2021,2022,와촌동,천안역사동아라이크텐,12,26,84.9864,충남 천안서북구,186,44133,38,,
3,"15,200",중개거래,1990,2022,성정동,현대1,12,4,71.03,충남 천안서북구,413-13,44133,9,,
4,"4,500",중개거래,2013,2022,성정동,평화로운나날,12,4,19.2875,충남 천안서북구,1347,44133,10,,


In [26]:
name_info = in_lawd + "_" + in_deal_ymd
name_info

'서북구_202212'

In [27]:
file_name = f"data/result_{name_info}.csv"
file_name

'data/result_서북구_202212.csv'

In [28]:
# 파일 저장
# result.to_csv(file_name, index=False)

##### 총 정리 함수

In [29]:
# 위를 총 정리하여 함수로 생성

def deal_by_lawd(cd, ymd):
    
    svckey = "fq448BTENSGVmu47zvzjDudVK1LUOPdIONYmOk774uKTYukGYTIZdMjVIURNLrwHFbDrRgCZRXehyB%2FR2YxQ5A%3D%3D"
    
    area_cd = "LAWD_CD=" + cd
    deal_dt = "DEAL_YMD=" + ymd
    svc_key = "serviceKey=" + svckey
    url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key


    response = requests.get(url)
    items_list = find_xml_items(response)
    result = pd.DataFrame(items_list)
    
    name_info = in_lawd + "_" + in_deal_ymd
    file_name = f"data/{name_info}.csv"

    result.to_csv(file_name, encoding='cp949', index=False)
    
    return file_name

In [30]:
#### - Input Data : LAWD_CD, DEAL_YMD, serviceKey  입력받기
in_lawd = input("검색지역 입력 : ")
print(in_lawd)

in_lawd_cd = find_lawd_cd(in_lawd)
print(in_lawd_cd)

in_deal_ymd = input("계약월 입력(예: 201911) : ")
print(in_deal_ymd)

검색지역 입력 : 수지구
수지구
41465
계약월 입력(예: 201911) : 202212
202212


In [31]:
deal_by_lawd(in_lawd_cd, in_deal_ymd)

'data/수지구_202212.csv'

In [32]:
pd.read_csv('./data/수지구_202212.csv', encoding ='cp949').head()

,거래금액,거래유형,건축년도,년,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"59,000",직거래,1999,2022,풍덕천동,현대성우8차,12,5,59.990,NaN,1112,41465,19,NaN,NaN
1,"63,000",중개거래,2000,2022,풍덕천동,신정마을우성,12,7,84.900,경기 용인수지구,1025,41465,5,NaN,NaN
2,"70,500",중개거래,2000,2022,풍덕천동,신정마을주공1,12,14,59.390,경기 용인수지구,1065,41465,7,NaN,NaN
3,"85,000",중개거래,2015,2022,풍덕천동,래미안수지이스트파크,12,14,84.980,경기 용인수지구,1198,41465,8,NaN,NaN
4,"108,500",직거래,2017,2022,풍덕천동,e편한세상수지,12,23,84.921,NaN,1209,41465,24,NaN,NaN
